In [2]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')
from tqdm import tqdm


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\user\anaconda3\envs\zoomcamp\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
user_question = "I just discovered the course. Can I still join it?"
len(embedding_model.encode(user_question))

768

Q1. Getting the embeddings model

What's the first value of the resulting vector?

In [5]:
embedding_model.encode(user_question)[0]

0.07822266

Prepare the documents
Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [9]:
len(documents)

948

In [12]:
mlops_documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]

In [13]:
len(mlops_documents)

375

Q2. Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
What's the shape of X? (X.shape). Include the parantheses.

In [26]:
#created the dense vector using the pre-trained model
emeddings = []
for doc in tqdm(mlops_documents):
    # Transforming the title into an embedding using the model
    vector = embedding_model.encode(f'{doc["question"]} {doc["text"]}').tolist()
    emeddings.append(vector)

In [34]:
embedding_model.encode(f'{doc["question"]} {doc["text"]}').tolist()

[0.04386020079255104,
 -0.01300770416855812,
 0.06373122334480286,
 0.03475850448012352,
 0.08711595833301544,
 -0.055447619408369064,
 -0.00405458640307188,
 0.032889705151319504,
 0.012904905714094639,
 -0.07416156679391861,
 -0.02473599649965763,
 -0.016438711434602737,
 0.09224607050418854,
 0.001718504005111754,
 0.0636737123131752,
 -0.03887335583567619,
 0.015595401637256145,
 -0.03959819674491882,
 -0.01618546061217785,
 -0.03351506590843201,
 -0.03041139245033264,
 -0.04308425635099411,
 0.02358408086001873,
 0.034822940826416016,
 -0.023112250491976738,
 0.0399431511759758,
 0.03210946172475815,
 -0.04315083101391792,
 0.0033354852348566055,
 -0.038175079971551895,
 0.020869886502623558,
 -0.031147966161370277,
 0.06125687435269356,
 0.0026883610989898443,
 -0.010857798159122467,
 -0.003777111414819956,
 0.011310724541544914,
 -0.0294178519397974,
 -0.02235438860952854,
 -0.03615126386284828,
 0.029324788600206375,
 0.011148547753691673,
 0.02369098551571369,
 0.0381251052021

In [35]:
import numpy as np
X = np.array(emeddings)

In [36]:
X.shape

(375, 768)

Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

In [37]:
from numpy.linalg import norm

In [52]:
def cosine_distance(vector, matrix, normalized = False):
    
    
    if normalized == False:
        # Нормализация вектора
        vector = vector / np.linalg.norm(vector)

        # Нормализация матрицы (каждой строки)
        matrix = matrix / np.linalg.norm(matrix, axis=1)[:, np.newaxis]
    
    # Вычисление косинусного расстояния (1 - косинусное сходство)
    cosine_similarity = np.dot(matrix, vector)
    cosine_distance = 1 - cosine_similarity
    
    return cosine_distance

# Пример использования
vector = np.array([1, 2, 3])
matrix = np.array([
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

distances = cosine_distance(vector, matrix, normalized= False)
print(distances)

[0.02536815 0.04058805 0.04874169]


In [57]:
v = embedding_model.encode(user_question)
distances = cosine_distance(v, X, normalized = True)
max(distances)

1.1467637144447336

In [58]:
scores = X.dot(v)

In [60]:
max(scores)

0.6506573301018632

Vector search
We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [61]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=mlops_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'text_vector': [0.0806286484003067,
   -0.06663887202739716,
   0.025273052975535393,
   -0.013004404492676258,
   0.07587281614542007,
   -0.05946270003914833,
   -0.021883828565478325,
   0.0029000297654420137,
   0.0007928494596853852,
   -0.005222412757575512,
   -0.03365180641412735,
   -0.027913479134440422,
   0.058116573840379715,
   0.03974832966923714,
   0.05441853776574135,
   -0.03825170546770096,
   0.063057370483

In [62]:
len(search_engine.search(v, num_results=5))

5

Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

In [63]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5

In [64]:
from sklearn.metrics import accuracy_score

In [67]:
[doc for doc in ground_truth if doc['document'] =='0a278fb2']

[{'question': 'What is the initial step after joining the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0a278fb2'},
 {'question': 'How can I view the content of the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0a278fb2'},
 {'question': 'Where can I find the materials related to my cohort?',
  'course': 'machine-learning-zoomcamp',
  'document': '0a278fb2'},
 {'question': 'How do I access the videos from previous office hours?',
  'course': 'machine-learning-zoomcamp',
  'document': '0a278fb2'},
 {'question': 'When was ML Zoomcamp first introduced?',
  'course': 'machine-learning-zoomcamp',
  'document': '0a278fb2'}]

In [78]:
hits = []
for doc in tqdm(ground_truth):
    v = embedding_model.encode(f'{doc["question"]}').tolist()
    results = search_engine.search(v, num_results=5)
    ids = [res['id'] for res in results]
    if doc['document'] in ids:
        hits.append(1)
    else:
        hits.append(0)

100%|██████████| 1830/1830 [01:36<00:00, 19.03it/s]


In [79]:
np.mean(hits)

0.9398907103825137

Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
Index the embeddings (note: you've already computed them)
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [82]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '1d8a531b3088', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'wcmjifaxRKCU3wqzMl_bag', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [86]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [85]:
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

dict_keys(['text', 'section', 'question', 'course', 'id', 'text_vector'])

In [88]:
len(results)

5

In [89]:
elastic_indexing = []
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text
    
    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)
    elastic_indexing.append(doc)

100%|██████████| 375/375 [03:26<00:00,  1.82it/s]


In [92]:
for doc in tqdm(elastic_indexing):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:28<00:00, 13.16it/s]


In [98]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [100]:
_ = elastic_search(
    query=user_question,
    #course="data-engineering-zoomcamp"
)

In [105]:
_[0]['id']

'ee58a693'

Q6. Hit-rate for Elasticsearch
The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?